# 2. Sequence 단위 RNN(Char RNN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

## 1. 훈련 데이터 전처리하기

In [2]:
#sentence = ("if you want to build a ship, don't drum up people together to "
#            "collect wood and don't assign them tasks and work, but rather "
#            "teach them to long for the endless immensity of the sea.")

In [3]:
sentence = "Repeat is the best medicine for memory".split()

In [4]:
vocab = list(set(sentence))
print(vocab)

['is', 'the', 'medicine', 'best', 'memory', 'Repeat', 'for']


In [5]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [6]:
print(word2index)

{'is': 1, 'the': 2, 'medicine': 3, 'best': 4, 'memory': 5, 'Repeat': 6, 'for': 7, '<unk>': 0}


In [7]:
print(word2index['memory'])

5


In [8]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'is', 2: 'the', 3: 'medicine', 4: 'best', 5: 'memory', 6: 'Repeat', 7: 'for', 0: '<unk>'}


In [9]:
print(index2word[2])

the


In [10]:
# Char data와 다르게 input과 output을 하나로 구성
# => Sequence data이기 때문이다

def build_data(sequence, word2index):
  encoded = [word2index[token] for token in sentence]  # 각 문자를 정수로 변환
  input_seq, label_seq = encoded[:-1], encoded[1:]  # 입력 시퀀스와 레이블 시퀀스를 분리
  input_seq = torch.LongTensor(input_seq).unsqueeze(0)  # 배치 차원추가(batch size를 위한)  # Longtensor? char 단위일때는 flaot/ sequence 단위일 때는 Long?
  label_seq = torch.LongTensor(label_seq).unsqueeze(0)  # 배치 차원 추가

  return input_seq, label_seq

In [11]:
X, Y = build_data(sentence, word2index)

In [12]:
print(X)
print(Y)

tensor([[6, 1, 2, 4, 3, 7]])
tensor([[1, 2, 4, 3, 7, 5]])


In [ ]:
len(word2index) 

8

# 2. 모델구현하기

https://dgkim5360.tistory.com/entry/deep-learning-for-nlp-with-pytorch-tutorial-part-2

In [ ]:
# 시퀀스 길이 = 6 / 7 => 6 앞 뒤에서 하나씩 뺐기 때문이다 / encoded[:-1], encoded[1:]
# 임베딩 차원 = 5 / 임베딩 차원이 5개라는 것은 관계지을 수 있는 단어의 개수가 5개라는 뜻이다
# 은닉층 크기 = 5 / 임베딩 차원과 동일하게?
# vocab_size = 8 

# [5, 4, 1, 6, 2, 3]

class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩 / vocab_size = real input size
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기(vocab_size)) (1, 6, 8)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기) (1 * 6, 8)
        return output.view(-1, output.size(2))

In [ ]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20

In [ ]:
len(word2index)

8

In [ ]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first = True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss()  # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨
# 옵티마이저 정의
optimizer = optim.Adam(params = model.parameters())

In [ ]:
# 임의로 예측해보기, 가중치는 전부 랜덤 초기화된 상태이다.
output = model(X)
print(output)

tensor([[-0.0022, -0.1335,  0.0503, -0.2161, -0.2643, -0.1881, -0.1972, -0.1656],
        [-0.1385, -0.2568, -0.0525, -0.2104, -0.4541, -0.2294,  0.1212, -0.3116],
        [ 0.1584, -0.3052, -0.0091, -0.2901, -0.1001, -0.2770, -0.0428, -0.1306],
        [ 0.0217, -0.1530, -0.0619, -0.1706, -0.3434, -0.2223, -0.1040, -0.0685],
        [-0.0705, -0.1156, -0.1755,  0.0048, -0.4573, -0.0704, -0.4216, -0.1748],
        [-0.0162, -0.1154, -0.2930,  0.0753, -0.2878, -0.0013, -0.3668,  0.0212]],
       grad_fn=<ViewBackward>)


In [ ]:
print(output.shape)  # (배치 크기*시퀀스 길이, 단어장 크기) / (1 * 6, 8) / < unk > 까지 포함

torch.Size([6, 8])


In [ ]:
torch.Size([6,8])

torch.Size([6, 8])

In [ ]:
# 수치화된 데이터를 단어로 전호나하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [ ]:
decode

<function __main__.<lambda>>

# 2. 훈련 시작

In [ ]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0391 
Repeat medicine best <unk> <unk> for for

[41/201] 1.4054 
Repeat medicine the best medicine for memory

[81/201] 0.7707 
Repeat is the best medicine for memory

[121/201] 0.3770 
Repeat is the best medicine for memory

[161/201] 0.1917 
Repeat is the best medicine for memory

[201/201] 0.1120 
Repeat is the best medicine for memory

